In [1]:
import numpy as np
import torch
from torch.autograd import Variable
from torch import nn
import loader
from threes import *
from qLearningNet import *
import time

MOVES = [0, 1, 2, 3]
CUDA = False

FILENAME = "saved_parameters"
INPUT_SIZE = 19
HIDDEN_SIZE = 256
GAMMA = 0.8


# parametry sieci np. learning rate
# lepszy zbior uczacy - jak w DeepMindzie
# nagrody - musza byc wieksze - inaczej sami sobie redukujemy learning rate
# (gestsze nagrody - np. logarytm z rewardow)
# uczyc jednak dobrymi przebiegami
# http://pytorch.org/docs/master/nn.html#embedding 

In [ ]:
def train(model, data_loaders, optimizer, num_epochs=500, log_every=100, verbose=True):
    if CUDA:
        model.network.cuda()

    data = np.zeros((1000,40))
    data_ptr = 0
    
    iter_ = 0
    epoch = 0
    if verbose:
        print u'Training the model!'
        print u'Interrupt at any time to get current model'
    try:
        while epoch < num_epochs:
            epoch += 1
            x = data_loaders.get(model, 30)
            #data[data_ptr:data_ptr + 30] = x
            #data_ptr += 30
            #data_ptr %= 1000 - 30
            
            #random_data_ptr = np.random.randint(0,data_ptr)
            #x = data[random_data_ptr : random_data_ptr + 30]
            
            future = x[:, 21:]
            future_scores = model.Q(future)
            for i, row in enumerate(future):
                #print(row)
                game = Threes(save_game=False, data=row.tolist())
                for j, move in enumerate(MoveEnum):
                    if not game.canMove(move):
                        future_scores[i, j] = float('-inf')
                if not game.getPossibleMoves():
                    future_scores[i:,:] = np.full((1,4), x[i, 20])
                
            y = x[:, 20] + GAMMA * np.max(future_scores, axis=1)
            xx = x[:, :19]
            iter_ += 1

            optimizer.zero_grad()
            i = np.asarray(np.vstack((np.arange(0,30),x[:,19])),int)
            
            i = torch.LongTensor(i)
            out = model.Q(xx, as_variable=True)[i[0],i[1]]
            loss = model.loss(out, y)
            loss.backward()
            optimizer.step()

            if iter_ % log_every == 0 and verbose:
                print u"Minibatch {0: >6}  | loss {1: >15.12f} ".format(iter_, loss.data[0])
            if iter_ % (log_every * 20) == 0:
                model.save_parameters(FILENAME)
                

    except KeyboardInterrupt:
        pass
    model.save_parameters(FILENAME)


q_learning_net = QLearningNet()
for p in q_learning_net.network.parameters():
    p.requires_grad = True
optimizer = torch.optim.Adam(q_learning_net.network.parameters(), lr=5.000)
data_loader = loader.Loader()
train(q_learning_net, data_loader, optimizer, num_epochs=400000)
print("Learning done")

Training the model!
Interrupt at any time to get current model
Minibatch    100  | loss 203.813262939453 
Minibatch    200  | loss 104.367820739746 
Minibatch    300  | loss 73869.093750000000 
Minibatch    400  | loss 649.117919921875 
Minibatch    500  | loss 1622.707885742188 
Minibatch    600  | loss 19754.361328125000 
Minibatch    700  | loss 2482.094970703125 
Minibatch    800  | loss 356.112457275391 
Minibatch    900  | loss 26.915170669556 
Minibatch   1000  | loss 2112.125732421875 
Minibatch   1100  | loss 149995.171875000000 
Minibatch   1200  | loss 21199.302734375000 
Minibatch   1300  | loss 536.539245605469 
Minibatch   1400  | loss 14.363482475281 
Minibatch   1500  | loss  2.086886644363 
Minibatch   1600  | loss 2054.673828125000 
Minibatch   1700  | loss 596.674804687500 
Minibatch   1800  | loss 5282.841796875000 
Minibatch   1900  | loss 75.195236206055 
Minibatch   2000  | loss 907.602172851562 
Minibatch   2100  | loss 46557.066406250000 
Minibatch   2200  | lo

In [3]:
from AIModels import *
game = Threes(save_game=False)
ai = QLearningNetAI(game, FILENAME)
scores, move_count, highs = AIModel.test_ai(ai, 1000, verbose=False)
print move_count
print np.mean(scores), np.mean(highs)

Loading net parameters
{'Down': 12518, 'Right': 1015, 'Up': 0, 'Left': 37135}
387.396 37.788


In [5]:
game = Threes(save_game=False)
ai = RandomAI(game)
scores, move_count, highs = AIModel.test_ai(ai, 1000, verbose=False)
print move_count
print np.mean(scores), np.mean(highs)

{'Down': 10711, 'Right': 10507, 'Up': 10646, 'Left': 10809}
288.54 33.432


In [11]:
game = Threes(save_game=False)
ai = MiniMaxAI(game)
scores, move_count, highs = AIModel.test_ai(ai, 10, verbose=False)
print move_count
print np.mean(scores), np.mean(highs)

{'Down': 775, 'Right': 741, 'Up': 761, 'Left': 779}
12808.5 412.8
